<a href="https://colab.research.google.com/github/salarbalou/photothermal_heating/blob/main/Photothermal_Study_of_CQDs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import plotly.io as pio
pio.templates.default= 'plotly_dark'
from plotly.subplots import make_subplots

In [53]:
CQD = ['red', 'orange', 'yellow', 'green', 'cyan', 'blue', 'violet', 'white']
Conc = ['10x', '5x', '2.5x', '0.75x'] # make 4 solutions of wach with different concentrations 
dc = np.arange(20, 150, 20) # contorlled by arduino

In [54]:
!wget 'https://raw.githubusercontent.com/salarbalou/steadystate_analysis/main/10-21-21-CQD-10x-dc90.txt'

--2021-11-23 16:33:31--  https://raw.githubusercontent.com/salarbalou/steadystate_analysis/main/10-21-21-CQD-10x-dc90.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4677 (4.6K) [text/plain]
Saving to: ‘10-21-21-CQD-10x-dc90.txt.1’

10-21-21-CQD-10x-dc 100%[===================>]   4.57K  --.-KB/s    in 0s      

2021-11-23 16:33:31 (32.0 MB/s) - ‘10-21-21-CQD-10x-dc90.txt.1’ saved [4677/4677]



In [3]:
 data = np.loadtxt('https://raw.githubusercontent.com/salarbalou/steadystate_analysis/main/10-21-21-CQD-10x-dc90.txt', delimiter=' , ')
 data = data[23:]

In [4]:
#for blue 435nm maxpower=4W laser:
m, b = 7.025524475524473, -66.07575757575763
# dc is arduino board pwm pin duty cycle

In [5]:
# *** Steady State Analysis *** Heating rate **** #turning on laser for 120 seconds to measure the heating rate
t = data[:, 0] #time (s)
T = data[:, 1] #temperature of the solution (C)
Tw = np.full(t.size, 22)#temperature of the environment (C) (needs to be measured by another K type thermocouple) assumed to be constant at 22 C
R  = np.gradient(T, t) #gradient of T with respect to time
HR = np.max(R) #heating rate
Teq = T[np.min(np.where(R<0.01))] #equilibrium temperature (C)
teq = t[np.min(np.where(R<0.01))] # time required to reach equilibrium (s)
print(f'Heated up to equilibrium eemperature of {Teq} in {teq} seconds')

Reached equilibrium eemperature of 94.14 in 81.0 seconds


In [ ]:
# *** Steady State Analysis *** Cooling rate **** #turning off laser for 120 seconds to measure the cooling rate
t = data[:, 0] #time (s)
T = data[:, 1] #temperature of the solution (C)
Tw = np.full(t.size, 22)#temperature of the environment (C) (needs to be measured by another K type thermocouple) assumed to be constant at 22 C
C  = np.gradient(T, t) #gradient of T with respect to time
CR = np.max(C) #heating rate
Teq = T[np.min(np.where(C<0.01))] #equilibrium temperature (C)
teq = t[np.min(np.where(C<0.01))] # time required to reach equilibrium (s)
print(f'Cooled down to equilibrium temperature of {Teq} in {teq} seconds')

In [51]:
# *** Photothermal conversion efficiency ***

#Iout = #measured using powermeter: f(CCQD, L_laser) (W)
#S_PCR_tube = #surface area of the solution at which laser beam is passing through:[PCR_tube, paper] (m**2)
#S_paper = #surface are of the paper at which laser beam is covering:[PCR_tube, paper]  (m**2)
#m = rho * V #mass of solution (Kg)

Iin = m*dc + b #input power of the laser: f(dc) (W)
h = 1000 #heat transfor coefficient of solution with its surrounding (W/(m**2)*C)

rho  = 1000 #density of solution (kg/m**3)
V = 1e-8 #volume of the solution (m**3)
Cp = 4182 #heat capacity of water (J/Kg*C)
#eta = ((R*(Cp*m)+np.sum((h*S*(T - Tw))))/(Iin - Iout)
#print(f'Photothermal Conversion efficiency of your CQD is {eta}')

In [55]:
# *** what we need to know of each CQD***
#print(f' {Color} CQD reached Teq of {Teq} at {Iout} mW in {teq} seconds, eta: {eta}')

In [56]:
# ***Modeling the heat diffusion in PCR-Tube and paper***
nx = 2
x = np.arange(nx)
nt = np.int(teq) #temperature rise is evaluated in {t.size} seconds
Tplot = np.full(T.size, np.min(Tw))

In [57]:
fig = make_subplots()
fig.add_scatter(x = x, y = Tplot, name = '0 s')
for i in range (nt):
  Tplot[0] = T[i]
  Tplot[-1] = np.min(Tw)
  fig.add_scatter(x = np.arange(nx), y = Tplot, name = f'{i+1} s')
fig.update_layout(height = 800, width = 1100)
fig.show()